# 14. MongoDB & SECURITY

_(50min)_

1. [Security](https://www.mongodb.com/docs/ops-manager/current/tutorial/nav/security/#security)

## 14.1 Module introduction

- O módulo trata dos 3 primeiros blocos:

    <img src="..\imgs\s14\s14-1.png" width=600 height=300 >

---

## 14.2 Understanding role based access control

1. [Configure MongoDB Authentication and Authorization](https://www.mongodb.com/docs/ops-manager/current/tutorial/edit-host-authentication-credentials/#configure-mongodb-authentication-and-authorization)

<img src="..\imgs\s14\s14-2.png" width=700 height=400 >

- Role based access control:

    <img src="..\imgs\s14\s14-3.png" width=700 height=400 >

---

## 14.3 Roles - Examples

1. [Manage Custom Roles](https://www.mongodb.com/docs/ops-manager/current/tutorial/manage-mongodb-roles/#manage-custom-roles)

<img src="..\imgs\s14\s14-4.png" width=700 height=400 >

---

## 14.4 Creating a User

1. [Manage MongoDB Users](https://www.mongodb.com/docs/ops-manager/current/tutorial/manage-mongodb-users/#manage-mongodb-users)
2. [createUser](https://www.mongodb.com/docs/manual/reference/command/createUser/#createuser)
3. [db.createUser()](https://www.mongodb.com/docs/manual/reference/method/db.createUser/#db.createuser--)
4. [`db.auth()`](https://www.mongodb.com/docs/manual/reference/method/db.auth/#db.auth--)

> `createUser`: creates a new user on the database where you run the command. The `createUser` command returns a duplicate user error if the user exists.

>`db.createUser(user, writeConcern)`:
>
> Creates a new user for the database on which the method is run. `db.createUser()` returns a duplicate user error if the user already exists on the database.

| Field          | Type     | Description                                                                                                                                                                                                                                                         |
| -------------- | -------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `user`         | document | The document with authentication and access information about the user to create.                                                                                                                                                                                   |
| `writeConcern` | document | Optional. The level of [write concern](https://www.mongodb.com/docs/manual/reference/write-concern/#std-label-write-concern) for the operation. See [Write Concern Specification.](https://www.mongodb.com/docs/manual/reference/write-concern/#std-label-wc-specs) |


<img src="..\imgs\s14\s14-5.png" width=700 height=400 >

- [`db.auth()`](https://www.mongodb.com/docs/manual/reference/method/db.auth/#db.auth--)

In [ ]:
// use admin

db.createUser( { user: "Max", pwd: "max", roles: [ "userAdminAnyDatabase" ] })

db.auth("Max", "max")

----

## 14.5 Built-in roles - An overview

1.[Built-In Roles](https://www.mongodb.com/docs/manual/reference/built-in-roles/#built-in-roles)

> A role grants privileges to perform sets of [actions](https://www.mongodb.com/docs/manual/reference/privilege-actions/#privilege-actions) on defined [resources](https://www.mongodb.com/docs/manual/reference/resource-document/#resource-document-on-self-managed-deployments).<br>A given role applies to the database on which it is defined and can grant access down to a collection level of granularity.

<img src="..\imgs\s14\s14-6.png" width=600 height=300 >

---

## 14.6 Assigning roles to Users and Databases

- Logar no servidor já conectado a um usuário criado:

In [ ]:
mongosh -u Max -p max --authenticationDatabase admin

- Criar um novo usuário e autenticar-se no banco com ele:

In [ ]:
// use shop

db.createUser( { user: 'appdev', pwd: 'dev', roles: [ "readWrite" ]}) // { ok: 1 }

db.auth('appdev', 'dev') // { ok: 1 }

In [ ]:
show users

[
    {
      _id: 'shop.appdev',
      userId: UUID('76c884e3-3cbe-49ba-a0d2-525a7bb3f2a2'),
      user: 'appdev',
      db: 'shop',
      roles: [ { role: 'readWrite', db: 'shop' } ],
      mechanisms: [ 'SCRAM-SHA-1', 'SCRAM-SHA-256' ]
    }
  ]

- Inserir dados na coleção:

In [ ]:
db.products.insertOne( { name: "Book" }) // era para retornar um erro, pois mais de 1 usuário está autenticado (mas não aconteceu)

---

## 14.7 Updating and extending roles to Other databases

1. [updateUser](https://www.mongodb.com/docs/manual/reference/command/updateUser/#updateuser)
2. [`db.updateUser()`](https://www.mongodb.com/docs/manual/reference/method/db.updateUser/#db.updateuser--)

> `db.updateUser( username, update, writeConcern )`
>
> Updates the user's profile on the database on which you run the method.<br>
An update to a field completely replaces the previous field's values.<br>
This includes updates to the user's roles array.


- Atualizar a criação de um usuário:
    - as atualização não "incrementam" as `roles` do usuário, mas as substituem completamente
    - é possível dar acesso ao usuário a novas collections, incluindo uma nova `role` dentro das `roles` atuais, conforme exemplo

In [ ]:
db.updateUser("appdev", { roles: [ "readWrite", { role: "readWrite", db: "blog" } ]} )

- ocorre um erro pois foi tentado executar o comando logado com o usuário "appdev", que somente tem acesso "readWrite" à collection "shop"
- é necessário realizar o logout deste usuário e logar com o usuário admin "Max" para execução do comando

In [ ]:
db.logout()

use admin

db.auth("Max", "max")

In [ ]:
db.updateUser("appdev", { roles: [ "readWrite", { role: "readWrite", db: "blog" } ]} ) // { ok: 1 }

db.getUser()

- Resultado da busca `db.getUser()`:

In [ ]:
{
    _id: 'shop.appdev',
    userId: UUID('76c884e3-3cbe-49ba-a0d2-525a7bb3f2a2'),
    user: 'appdev',
    db: 'shop',
    roles: [
      { role: 'readWrite', db: 'shop' },
      { role: 'readWrite', db: 'blog' }
    ],
    mechanisms: [ 'SCRAM-SHA-1', 'SCRAM-SHA-256' ]
  }

---

## 14.8 Assignment 8: Time to Practice - Security

<img src="..\imgs\s14\s14-7.png" width=600 height=150 >

### 14.8.1 Database Admin

1. [userAdminAnyDatabase](https://www.mongodb.com/docs/manual/reference/built-in-roles/#mongodb-authrole-userAdminAnyDatabase)

In [ ]:
use admin # indo para o ambiente do admin

db.createUser({ 
    user: "Max", 
    pwd: "max", 
    roles: [ 
        "userAdminAnyDatabase" 
    ]
}) # criação do "super usuário"

db.auth("Max", "max") # autenticação com o usuário 

# mongo -u Max -p max --authenticationDatabase admin

### 14.8.2 User Admin

1. [dbAdminAnyDatabase](https://www.mongodb.com/docs/manual/reference/built-in-roles/#mongodb-authrole-dbAdminAnyDatabase)

In [ ]:
db.createUser({ 
    user: "globalAdmin", 
    pwd: "admin", 
    roles: [ 
        "dbAdminAnyDatabase" 
    ]
}) # criação do usuário admin

# mongo -u globalAdmin -p admin --authenticationDatabase admin

### 14.8.3 Developer

In [ ]:
db.createUser({ 
    user: "dev", 
    pwd: "dev", 
    roles: [ 
        { role: "readWrite", db: "customers"},
        { role: "readWrite", db: "sales"} 
    ]
}) # criação do usuário developer com acesso à 2 collections

# mongo -u dev -p dev --authenticationDatabase admin

---

## 14.9 Adding SSL Transport Encryption (_deprecated_)

1. [Configure mongod and mongos for TLS/SSL](https://www.mongodb.com/docs/manual/tutorial/configure-ssl/#configure-mongod-and-mongos-for-tls-ssl)
2. [Upgrade a Cluster to Use TLS/SSL](https://www.mongodb.com/docs/manual/tutorial/upgrade-cluster-to-ssl/#upgrade-a-cluster-to-use-tls-ssl)
3. [TLS/SSL (Transport Encryption)](https://www.mongodb.com/docs/manual/core/security-transport-encryption/#tls-ssl--transport-encryption-)


<img src="..\imgs\s14\s14-8.png" width=500 height=150 >

- [openssl/binaries](https://github.com/openssl/openssl/wiki/Binaries) -> OpenSSL for Windows
- [Download Win32/Win64 OpenSSL today using the links below!](https://slproweb.com/products/Win32OpenSSL.html) -> Win64 OpenSSL v3.5.0 Light

- [How can I generate a self-signed SSL certificate using OpenSSL? [closed]](https://stackoverflow.com/questions/10175812/how-can-i-generate-a-self-signed-ssl-certificate-using-openssl)


1. Em um cmd, navegar até a pasta "OpenSSL-Win64\bin" e inserior o comando abaixo:

    `openssl req -newkey rsa:2048 -new -x509 -days 365 -nodes -out mongodb-cert.crt -keyout mongodb-cert.key`

In [ ]:
C:\Program Files\OpenSSL-Win64\bin>openssl req -newkey rsa:2048 -new -x509 -days 365 -nodes -out mongodb-cert.crt -keyout mongodb-cert.key
.....+....+..+....+++++++++++++++++++++++++++++++++++++++*.........+.+..+...+......+++++++++++++++++++++++++++++++++++++++*...+....+...........+...+............+.........+.+.........+.....+.........++++++
...+....+...+.....+.......+..+.+++++++++++++++++++++++++++++++++++++++*.....+......+............+..+...+......+.+.........+.........+..+......+...+.......+.....+.......+......+......+...+++++++++++++++++++++++++++++++++++++++*......+......+.+........+.+........+.+......+......+..+......+......+....+.....+....+...+...+..+............+...+...+.+........+.......+...+..+...+.+...............+..+.......+.....+......+......+.+...............+.....+....+...........+....+......+..+...+...+....+..+...+.......++++++
-----
You are about to be asked to enter information that will be incorporated
into your certificate request.
What you are about to enter is what is called a Distinguished Name or a DN.
There are quite a few fields but you can leave some blank
For some fields there will be a default value,
If you enter '.', the field will be left blank.
-----
Country Name (2 letter code) [AU]:
State or Province Name (full name) [Some-State]:
Locality Name (eg, city) []:
Organization Name (eg, company) [Internet Widgits Pty Ltd]:
Organizational Unit Name (eg, section) []:
Common Name (e.g. server FQDN or YOUR name) []:localhost # no caso de uma aplicação real, aqui deve-se colocar o endereço do servidor de conexão
Email Address []:max@test.com

2. Utilizar o comando abaixo para gerar as chaves e certificados:

    `type mongodb-cert.key mongodb-cert.crt > mongodb.pem`

In [ ]:
mongodb-cert.key
mongodb-cert.crt

3. Copiar o conteúdo do certificado:

    `copy mongodb-cert.crt mongodb-ca.crt`

4. Conectar à instância com um dos comandos abaixo:

    `mongod --tlsMode requireTLS --tlsCertificateKeyFile mongodb.pem --tlsCAFile mongodb-ca.crt`

    `mongod --tlsMode requireTLS --tlsCertificateKeyFile mongodb.pem --tlsCAFile mongodb-ca.crt --tlsAllowInvalidCertificates`

> NENHUM DOS COMANDOS OBTEVE ÊXITO NO WINDOWS, REVISAR O CONTEÚDO DESTE CAPÍTULO NOVAMENTE, SEM SEGUIR O CURSO.

---

## 14.10 Encryption at REST

1. [Encryption at Rest](https://www.mongodb.com/docs/manual/core/security-encryption-at-rest/#encryption-at-rest)

<img src="..\imgs\s14\s14-9.png" width=550 height=300 >

---

## 14.11 Wrap up

<img src="..\imgs\s14\s14-10.png" width=700 height=350 >

---

# 15.12 Useful resources and links

> Helpful Articles/ Docs:
>
> - Official "Encryption at Rest" Docs: https://docs.mongodb.com/manual/core/security-encryption-at-rest/
> - Official Security Checklist: https://docs.mongodb.com/manual/administration/security-checklist/
> - What is SSL/ TLS? => https://www.acunetix.com/blog/articles/tls-security-what-is-tls-ssl-part-1/
> - Official MongoDB SSL Setup Docs: https://docs.mongodb.com/manual/tutorial/configure-ssl/
> - Official MongoDB Users & Auth Docs: https://docs.mongodb.com/manual/core/authentication/
> - Official Built-in Roles Docs: https://docs.mongodb.com/manual/core/security-built-in-roles/
> - Official Custom Roles Docs: https://docs.mongodb.com/manual/core/security-user-defined-roles/